In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import *
import cv2
import sklearn
import matplotlib.pyplot as plt
import os
tf.compat.v1.disable_eager_execution()

In [2]:
tf.compat.v1.reset_default_graph()

In [3]:
np.random.seed(42)
tf.compat.v1.set_random_seed(42)

In [4]:
train_path = "../input/p800-data/224Frames/train"
val_path = "../input/p800-data/224Frames/val"
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rotation_range= 45,horizontal_flip = True)
val_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rotation_range= 45,horizontal_flip = True)

In [5]:
train_gen = train_data_generator.flow_from_directory(train_path,target_size = (256,256),color_mode = "rgb",class_mode = "categorical",batch_size = 100,shuffle = True)

Found 11707 images belonging to 2 classes.


In [6]:
val_gen = val_data_generator.flow_from_directory(val_path,target_size = (256,256),color_mode = "rgb",class_mode = "categorical",batch_size = 100,shuffle = True)

Found 2507 images belonging to 2 classes.


In [7]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps*11*11
caps1_n_dims = 8
caps2_n_caps = 2
caps2_n_dims = 16

In [8]:
conv1_p = {
    "strides": 2,
    "filters": 64,
    "padding":"VALID",
    "activation": "relu",
    "kernel_size": 4
}
conv2_p = {
    "strides": 2,
    "filters": 128,
    "padding":"VALID",
    "activation": "relu",
    "kernel_size": 6
}
conv3_p = {
    "strides": 2,
    "filters": 256,
    "padding":"VALID",
    "activation": "relu",
    "kernel_size": 6
}
conv4_p = {
    "strides": 2,
    "filters": 256,
    "padding":"VALID",
    "activation": "relu",
    "kernel_size": 8
}
conv1_params = {
    "strides": 1,
    "filters": 256,
    "padding":"VALID",
    "activation": "relu",
    "kernel_size": 9
}
conv2_params = {
    "strides": 2,
    "kernel_size":9,
    "filters": caps1_n_maps*caps1_n_dims,
    "padding":"VALID",
    "activation": "relu"
}

In [9]:
X = tf.compat.v1.placeholder(shape = (None,256,256,3),dtype =tf.float32)

In [10]:
conv1 = tf.compat.v1.layers.conv2d(X,**conv1_p)
conv2 = tf.compat.v1.layers.conv2d(conv1,**conv2_p)
conv3 = tf.compat.v1.layers.conv2d(conv2,**conv3_p)
conv4 = tf.compat.v1.layers.conv2d(conv3,**conv4_p)
conv2_reshaped = tf.reshape(conv4,shape= (-1,caps1_n_caps,caps1_n_dims))
print(conv1)
print(conv2)
print(conv3)
print(conv4)
print(conv2_reshaped)

Tensor("conv2d/Relu:0", shape=(None, 127, 127, 64), dtype=float32)
Tensor("conv2d_1/Relu:0", shape=(None, 61, 61, 128), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(None, 28, 28, 256), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(None, 11, 11, 256), dtype=float32)
Tensor("Reshape:0", shape=(None, 3872, 8), dtype=float32)


In [11]:
def squash(s,axis = -1,epsilon = 1e-9):
    with tf.compat.v1.name_scope("Squashing"):
        square = tf.reduce_sum(tf.square(s),axis = axis,keepdims = True)
        norm = tf.sqrt(square+epsilon)
        squashed = square/(1.+square)
        squashed = squashed/norm
        return s*squashed

In [12]:
caps1_output= squash(conv2_reshaped)

In [13]:
W_init = tf.compat.v1.random_normal(shape = (1,caps1_n_caps,caps2_n_caps,caps2_n_dims,caps1_n_dims))
W = tf.Variable(W_init,name = "WeightMatrix")
batch_shape = tf.shape(X)[0]
W_tiled = tf.tile(W,[batch_shape,1,1,1,1],name = "W_tiled")

print(W_tiled)

caps1_output_exp = tf.expand_dims(caps1_output,axis = -1)
caps1_output_exp = tf.expand_dims(caps1_output_exp,axis = 2)
caps1_output_ = tf.tile(caps1_output_exp,[1,1,caps2_n_caps,1,1])

print(caps1_output_)

caps_predicted = tf.matmul(W_tiled,caps1_output_)

print(caps_predicted)

Tensor("W_tiled:0", shape=(None, 3872, 2, 16, 8), dtype=float32)
Tensor("Tile:0", shape=(None, 3872, 2, 8, 1), dtype=float32)
Tensor("MatMul:0", shape=(None, 3872, 2, 16, 1), dtype=float32)


In [14]:
b_j = tf.zeros(shape =[tf.shape(X)[0],caps1_n_caps,caps2_n_caps,1,1],dtype = tf.float32)
c_j = tf.nn.softmax(b_j,axis = 2)

In [15]:
weighted_prediction = tf.multiply(c_j,caps_predicted)
print(weighted_prediction)
weighted_sum = tf.reduce_sum(weighted_prediction,axis = 1,keepdims = True)
print(weighted_sum)
caps2_output_round_1 = squash(weighted_sum,axis = -2)
print(caps2_output_round_1)

Tensor("Mul:0", shape=(None, 3872, 2, 16, 1), dtype=float32)
Tensor("Sum:0", shape=(None, 1, 2, 16, 1), dtype=float32)
Tensor("Squashing_1/mul:0", shape=(None, 1, 2, 16, 1), dtype=float32)


In [16]:
caps2_output_round2 = tf.tile(caps2_output_round_1,[1,caps1_n_caps,1,1,1])
print(caps2_output_round2)
print(caps_predicted)
agreement = tf.matmul(caps_predicted,caps2_output_round2,transpose_a = True)
print(agreement)

Tensor("Tile_1:0", shape=(None, 3872, 2, 16, 1), dtype=float32)
Tensor("MatMul:0", shape=(None, 3872, 2, 16, 1), dtype=float32)
Tensor("MatMul_1:0", shape=(None, 3872, 2, 1, 1), dtype=float32)


In [17]:
raw_weight_round_2 = tf.add(agreement,b_j)
softmax_round_2_b_j = tf.nn.softmax(raw_weight_round_2,axis = 2)
predicted_output_round2 = tf.multiply(softmax_round_2_b_j,caps_predicted)
print(predicted_output_round2)
predicted_sum_round2 = tf.reduce_sum(predicted_output_round2,axis = 1 ,keepdims = True)
print(predicted_sum_round2)
predicted_squashed_round2 =  squash(predicted_sum_round2,axis = -2)
print(predicted_sum_round2)
caps2_output = predicted_squashed_round2

Tensor("Mul_1:0", shape=(None, 3872, 2, 16, 1), dtype=float32)
Tensor("Sum_1:0", shape=(None, 1, 2, 16, 1), dtype=float32)
Tensor("Sum_1:0", shape=(None, 1, 2, 16, 1), dtype=float32)


In [18]:
def safe_norm(s, axis=-1, epsilon=1e-9, keep_dims=False, name=None):
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keepdims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [19]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

In [20]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")

In [21]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
y_pred

<tf.Tensor 'y_pred:0' shape=(None,) dtype=int64>

In [22]:
y = tf.compat.v1.placeholder(shape=[None], dtype=tf.int64, name="y")

In [23]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [24]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 2),
                           name="present_error")
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 2),
                          name="absent_error")
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")

In [25]:
alpha = 0.0005
loss = margin_loss
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")


optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001)
training_op = optimizer.minimize(loss, name="training_op")

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

In [26]:
n_epochs = 100
batch_size = 100
restore_checkpoint = True

n_iterations_per_epoch =100
n_iterations_validation = 25
best_loss_val = np.infty
checkpoint_path = "new"

with tf.compat.v1.Session() as sess:
    if restore_checkpoint and tf.compat.v1.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()

    for epoch in range(n_epochs):
        for iteration in range(1, n_iterations_per_epoch):
            # X_batch, y_batch = x_train[iteration*batch_size:(iteration+1)*batch_size],y_train_[iteration*batch_size:(iteration+1)*batch_size]
            X_batch, y_batch = next(train_gen)
            X_batch, y_batch = X_batch/255.0,np.argmax(y_batch,axis = 1)

            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 256, 256, 3]),
                           y: y_batch})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
        loss_vals = []
        acc_vals = []
        for iteration in range(1, n_iterations_validation + 1):
            # X_batch, y_batch = x_val[iteration*batch_size:(iteration+1)*batch_size],y_val_[iteration*batch_size:(iteration+1)*batch_size]
            X_batch, y_batch = next(val_gen)
            X_batch, y_batch = X_batch/255.0,np.argmax(y_batch,axis = 1)
            loss_val, acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={X: X_batch.reshape([-1, 256, 256, 3]),
                               y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch: 1  Val accuracy: 77.3200%  Loss: 0.153680 (improved)


ValueError: 'latest_filename' collides with 'save_path': 'checkpoint' and 'checkpoint'